In [ ]:
# This notebook has been updated to use the latest openai package version! At the time, 1.6.1

In [1]:
!poetry run python -V

Python 3.10.9


In [1]:
'''
Chapter 2: Launching an Application with Proprietary Models 
    Overview of Proprietary Models
    Introduction to OpenAI + Embeddings / GPT3 / ChatGPT
    Introduction to Vector Databases
    Building a Neural/Semantic Information Retrieval System with Vector Databases, BERT & GPT3

'''

'\nChapter 2: Launching an Application with Proprietary Models \n    Overview of Proprietary Models\n    Introduction to OpenAI + Embeddings / GPT3 / ChatGPT\n    Introduction to Vector Databases\n    Building a Neural/Semantic Information Retrieval System with Vector Databases, BERT & GPT3\n\n'

In [2]:
from openai import OpenAI
from datetime import datetime
import hashlib
import re
import os
from tqdm import tqdm
import numpy as np
from dotenv import load_dotenv

load_dotenv()

import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


In [3]:
pinecone_key = os.environ.get('PINECONE_API_KEY')
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

INDEX_NAME = 'semantic-search'
NAMESPACE = 'default'
ENGINE = 'text-embedding-ada-002'

In [4]:
# import pinecone

# pinecone.init(api_key=pinecone_key, environment="us-west1-gcp")

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

In [5]:
# helper functions to get lists of embeddings from the OpenAI API
def get_embeddings(texts, engine=ENGINE):
    response = client.embeddings.create(
        input=texts,
        model=engine
    )
    
    return [d.embedding for d in list(response.data)]

def get_embedding(text):
    return get_embeddings([text])[0]
    
len(get_embedding('hi')), len(get_embeddings(['hi', 'hello']))

(1536, 2)

In [6]:
pc.list_indexes().names()

[]

In [7]:
if not INDEX_NAME in pc.list_indexes().names():
    pc.create_index(
        INDEX_NAME,  # The name of the index
        dimension=1536,  # The dimensionality of the vectors
        metric='cosine',  # The similarity metric to use when searching the index
        # pod_type="p1"  # The type of Pinecone pod
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    )

# Store the index as a variable
index = pc.Index(INDEX_NAME)

In [8]:
def my_hash(s):
    # Return the MD5 hash of the input string as a hexadecimal string
    return hashlib.md5(s.encode()).hexdigest()

my_hash('I love to hash it')

'ae76cc4dfd345ecaeea9b8ba0d5c3437'

In [9]:
def prepare_for_pinecone(texts, engine=ENGINE):
    # Get the current UTC date and time
    now = datetime.utcnow()
    
    # Generate vector embeddings for each string in the input list, using the specified engine
    embeddings = get_embeddings(texts, engine=engine)
    
    # Create tuples of (hash, embedding, metadata) for each input string and its corresponding vector embedding
    # The my_hash() function is used to generate a unique hash for each string, and the datetime.utcnow() function is used to generate the current UTC date and time
    return [
        (
            my_hash(text),  # A unique ID for each string, generated using the my_hash() function
            embedding,  # The vector embedding of the string
            dict(text=text, date_uploaded=now)  # A dictionary of metadata, including the original text and the current UTC date and time
        ) 
        for text, embedding in zip(texts, embeddings)  # Iterate over each input string and its corresponding vector embedding
    ]


In [10]:
texts = ['hi']

In [11]:
prepare_for_pinecone(texts)[0]

('49f68a5c8493ec2c0bf489821c21fc3b',
 [-0.030913319438695908,
  -0.020414210855960846,
  -0.019505759701132774,
  -0.04178878664970398,
  -0.024813713505864143,
  0.024307576939463615,
  -0.0179743692278862,
  -0.017701834440231323,
  -0.0065019200555980206,
  -0.015910886228084564,
  0.025890879333019257,
  -0.006949656642973423,
  -0.01790948025882244,
  -0.011848808266222477,
  0.011465960182249546,
  0.01648191176354885,
  0.038751959800720215,
  0.0005187098286114633,
  0.03221110627055168,
  -0.008701670914888382,
  -0.019635537639260292,
  -0.0049056401476264,
  -0.009298654273152351,
  -0.014327583834528923,
  -0.022867031395435333,
  0.002483642427250743,
  0.010051371529698372,
  -0.01176445186138153,
  0.0026069325394928455,
  -0.026020657271146774,
  0.014535229653120041,
  0.0006987779634073377,
  -0.035767048597335815,
  -0.014963500201702118,
  -0.009486833587288857,
  -0.024748824536800385,
  0.006988590583205223,
  -0.02111501805484295,
  0.01918131299316883,
  -0.0056

In [12]:
_id, embedding, metadata = prepare_for_pinecone(texts)[0]

print('ID:  ',_id, '\nLEN: ', len(embedding), '\nMETA:', metadata)

ID:   49f68a5c8493ec2c0bf489821c21fc3b 
LEN:  1536 
META: {'text': 'hi', 'date_uploaded': datetime.datetime(2024, 6, 18, 7, 36, 6, 902370)}


In [13]:
texts = [
    "Alexander Graham Bell originally suggested 'ahoy-hoy' be adopted as the standard greeting when answering a telephone, before 'hello' (suggested by Thomas Edison) became common.",
    "The Abbott family -- wife Evelyn, husband Lee, congenitally deaf daughter Regan, and sons Marcus and Beau -- silently scavenge for supplies in a deserted town. While out in the open, the family communicates with American Sign Language (ASL). Four-year-old Beau is drawn to a battery-powered space shuttle toy, but Lee takes it away due to the noise it makes. Regan returns the toy to Beau, who also takes the batteries that his father removed from it. Beau activates the toy when the family is walking home and crossing a bridge, giving away his location to a nearby creature which kills him before Lee can save him.",
    "The second part of the novel begins with Pi's family aboard the Tsimtsum, a Japanese freighter that is transporting animals from their zoo to North America. A few days out of port from Manila, the ship encounters a storm and sinks. Pi manages to escape in a small lifeboat, only to learn that the boat also holds a spotted hyena, an injured Grant's zebra, and an orangutan named Orange Juice. Much to the boy's distress, the hyena kills the zebra and then Orange Juice. A tiger has been hiding under the boat's tarpaulin: it's Richard Parker, who had boarded the lifeboat with ambivalent assistance from Pi himself some time before the hyena attack. Suddenly emerging from his hideaway, Richard Parker kills and eats the hyena."
]

In [14]:
def upload_texts_to_pinecone(texts, namespace=NAMESPACE, batch_size=None, show_progress_bar=False):
    # Call the prepare_for_pinecone function to prepare the input texts for indexing
    total_upserted = 0
    if not batch_size:
        batch_size = len(texts)

    _range = range(0, len(texts), batch_size)
    for i in tqdm(_range) if show_progress_bar else _range:
        batch = texts[i: i + batch_size]
        prepared_texts = prepare_for_pinecone(batch)

        # Use the upsert() method of the index object to upload the prepared texts to Pinecone
        total_upserted += index.upsert(
            prepared_texts,
            namespace=namespace
        )['upserted_count']

    return total_upserted


# Call the upload_texts_to_pinecone() function with the input texts
upload_texts_to_pinecone(texts)


3

In [15]:
def query_from_pinecone(query, top_k=3):
    # get embedding from THE SAME embedder as the documents
    # query_embedding = get_embedding(query, engine=ENGINE)
    query_embedding = get_embedding(query)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      namespace=NAMESPACE,
      include_metadata=True   # gets the metadata (dates, text, etc)
    ).get('matches')

query_from_pinecone('hello')

[{'id': '093601540a641d12a6f734a9fa624ce5',
  'metadata': {'date_uploaded': '2024-06-18T07:36:10.203605',
               'text': "Alexander Graham Bell originally suggested 'ahoy-hoy' "
                       'be adopted as the standard greeting when answering a '
                       "telephone, before 'hello' (suggested by Thomas Edison) "
                       'became common.'},
  'score': 0.779527545,
  'values': []},
 {'id': '9588c26cecaaf486eae14858827a6699',
  'metadata': {'date_uploaded': '2024-06-18T07:36:10.203605',
               'text': 'The Abbott family -- wife Evelyn, husband Lee, '
                       'congenitally deaf daughter Regan, and sons Marcus and '
                       'Beau -- silently scavenge for supplies in a deserted '
                       'town. While out in the open, the family communicates '
                       'with American Sign Language (ASL). Four-year-old Beau '
                       'is drawn to a battery-powered space shuttle toy, b

In [16]:
import hashlib

def delete_texts_from_pinecone(texts, namespace=NAMESPACE):
    # Compute the hash (id) for each text
    hashes = [hashlib.md5(text.encode()).hexdigest() for text in texts]
    
    # The ids parameter is used to specify the list of IDs (hashes) to delete
    return index.delete(ids=hashes, namespace=namespace)

# delete our text
delete_texts_from_pinecone(texts)

# test that the index is empty
query_from_pinecone('hello')

[]

In [17]:
# Importing the tiktoken library
import tiktoken

# Initializing a tokenizer for the 'cl100k_base' model
# This tokenizer is designed to work with the 'ada-002' embedding model
tokenizer = tiktoken.get_encoding("cl100k_base")

# Using the tokenizer to encode the text 'hey there'
# The resulting output is a list of integers representing the encoded text
# This is the input format required for embedding using the 'ada-002' model
tokenizer.encode('hey there')


[36661, 1070]

In [18]:
# Function to split the text into chunks of a maximum number of tokens. Inspired by OpenAI
def overlapping_chunks(text, max_tokens = 500, overlapping_factor = 5):
    '''
    max_tokens: tokens we want per chunk
    overlapping_factor: number of sentences to start each chunk with that overlaps with the previous chunk
    '''

    # Split the text using punctuation
    sentences = re.split(r'[.?!]', text)

    # Get the number of tokens for each sentence
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    
    chunks, tokens_so_far, chunk = [], 0, []

    # Loop through the sentences and tokens joined together in a tuple
    for sentence, token in zip(sentences, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater 
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            if overlapping_factor > 0:
                chunk = chunk[-overlapping_factor:]
                tokens_so_far = sum([len(tokenizer.encode(c)) for c in chunk])
            else:
                chunk = []
                tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of 
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the sentence to the chunk and add the number of tokens to the total
        chunk.append(sentence)
        tokens_so_far += token + 1
    if chunk:
        chunks.append(". ".join(chunk) + ".")

    return chunks

In [19]:
import PyPDF2

# Open the PDF file in read-binary mode
with open('../data/pds2.pdf', 'rb') as file:

    # Create a PDF reader object
    reader = PyPDF2.PdfReader(file)

    # Initialize an empty string to hold the text
    principles_of_ds = ''
    # Loop through each page in the PDF file
    for page in tqdm(reader.pages):
        text = page.extract_text()
        principles_of_ds += '\n\n' + text[text.find(' ]')+2:]

# Print the final string containing all the text from the PDF file
principles_of_ds = principles_of_ds.strip()

print(len(principles_of_ds))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 428/428 [13:30<00:00,  1.89s/it]

575490


In [20]:
from urllib.request import urlopen

#

# A textbook about insects
text = urlopen('https://www.gutenberg.org/cache/epub/10834/pg10834.txt').read().decode()


In [21]:
split = overlapping_chunks(principles_of_ds, overlapping_factor=0)
avg_length = sum([len(tokenizer.encode(t)) for t in split]) / len(split)
print(f'non-overlapping chunking approach has {len(split)} documents with average length {avg_length:.1f} tokens')

non-overlapping chunking approach has 286 documents with average length 474.1 tokens


In [22]:
split = overlapping_chunks(principles_of_ds)
avg_length = sum([len(tokenizer.encode(t)) for t in split]) / len(split)
print(f'overlapping chunking approach has {len(split)} documents with average length {avg_length:.1f} tokens')

overlapping chunking approach has 392 documents with average length 485.2 tokens


In [23]:
# Importing the Counter and re libraries
from collections import Counter
import re

# Find all occurrences of one or more spaces in 'principles_of_ds'
matches = re.findall(r'[\s]{1,}', principles_of_ds)

# The 10 most frequent spaces that occur in the document
most_common_spaces = Counter(matches).most_common(10)

# Print the most common spaces and their frequencies
print(most_common_spaces)


[(' ', 82259), ('\n', 9220), ('  ', 1592), ('\n\n', 333), ('\n   ', 250), ('\n\n\n', 82), ('\n    ', 73), ('\n ', 46), (' \n', 39), ('     ', 34)]


In [24]:
# Only keep documents of at least 100 characters split by a custom delimiter
split = list(filter(lambda x: len(x) > 50, principles_of_ds.split('\n\n')))

avg_length = sum([len(tokenizer.encode(t)) for t in split]) / len(split)
print(f'custom delimiter approach has {len(split)} documents with average length {avg_length:.1f} tokens')

custom delimiter approach has 426 documents with average length 316.3 tokens


In [25]:
embeddings = None
for s in tqdm(range(0, len(split), 100)):
    if embeddings is None:
        embeddings = np.array(get_embeddings(split[s:s+100], engine=ENGINE))
    else:
        embeddings = np.vstack([embeddings, np.array(get_embeddings(split[s:s+100], engine=ENGINE))])
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.34s/it]


In [26]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assume you have a list of text embeddings called `embeddings`
# First, compute the cosine similarity matrix between all pairs of embeddings
cosine_sim_matrix = cosine_similarity(embeddings)

# Instantiate the AgglomerativeClustering model
agg_clustering = AgglomerativeClustering(
    n_clusters=None,         # the algorithm will determine the optimal number of clusters based on the data
    distance_threshold=0.1,  # clusters will be formed until all pairwise distances between clusters are greater than 0.1
    # affinity='precomputed',  # we are providing a precomputed distance matrix (1 - similarity matrix) as input
    linkage='complete'       # form clusters by iteratively merging the smallest clusters based on the maximum distance between their components
)

# Fit the model to the cosine distance matrix (1 - similarity matrix)
agg_clustering.fit(1 - cosine_sim_matrix)

# Get the cluster labels for each embedding
cluster_labels = agg_clustering.labels_

# Print the number of embeddings in each cluster
unique_labels, counts = np.unique(cluster_labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f'Cluster {label}: {count} embeddings')


Cluster 0: 1 embeddings
Cluster 1: 1 embeddings
Cluster 2: 1 embeddings
Cluster 3: 1 embeddings
Cluster 4: 1 embeddings
Cluster 5: 1 embeddings
Cluster 6: 1 embeddings
Cluster 7: 1 embeddings
Cluster 8: 1 embeddings
Cluster 9: 1 embeddings
Cluster 10: 1 embeddings
Cluster 11: 1 embeddings
Cluster 12: 1 embeddings
Cluster 13: 1 embeddings
Cluster 14: 1 embeddings
Cluster 15: 1 embeddings
Cluster 16: 1 embeddings
Cluster 17: 1 embeddings
Cluster 18: 1 embeddings
Cluster 19: 1 embeddings
Cluster 20: 1 embeddings
Cluster 21: 1 embeddings
Cluster 22: 1 embeddings
Cluster 23: 1 embeddings
Cluster 24: 1 embeddings
Cluster 25: 1 embeddings
Cluster 26: 1 embeddings
Cluster 27: 1 embeddings
Cluster 28: 1 embeddings
Cluster 29: 1 embeddings
Cluster 30: 1 embeddings
Cluster 31: 1 embeddings
Cluster 32: 1 embeddings
Cluster 33: 1 embeddings
Cluster 34: 1 embeddings
Cluster 35: 1 embeddings
Cluster 36: 1 embeddings
Cluster 37: 1 embeddings
Cluster 38: 1 embeddings
Cluster 39: 1 embeddings
Cluster 40

In [27]:
pruned_documents = []
for _label, count in zip(unique_labels, counts):
    pruned_documents.append('\n\n'.join([text for text, label in zip(split, cluster_labels) if label == _label]))

    
avg_length = sum([len(tokenizer.encode(t)) for t in pruned_documents]) / len(pruned_documents)
print(f'Our pruning approach has {len(pruned_documents)} documents with average length {avg_length:.1f} tokens')

Our pruning approach has 426 documents with average length 316.3 tokens


In [28]:
print(pruned_documents[0])

45, 349, 350
   survey  356, 357, 359, 360, 362, 363
Spark  376
spark_sklearn
   reference link  405
square matrix  77
standard deviation  146
standard normal distribution  133
statistical modeling  226
statistics  137, 201
   measures of center  144
   measures of relative standing  150, 153, 156
   measures of variation  145, 148
   measuring  144
stock price
   example  355
stock prices
   predicting, on social media  338
   text sentiment analysis  338, 339
structured data
   about  33
   versus unstructured data  33
superset  87
supervised learning  215
supervised learning models  271
supervised learning
   classification  219
   regression  219
   types  219
supervised machine learning  224
T
TensorFlow
   about  368, 369, 370, 371, 372, 374
   neural networks  368, 369, 370, 371, 372, 374   using  363, 364, 366, 367
titanic dataset  68
training error
   versus cross-validation error  318, 320
U
underfitting  308
unstructured data
   about  33
   versus structured data  33
unsupe

In [29]:
upload_texts_to_pinecone(pruned_documents, batch_size=128)

426

In [30]:
query = 'How do z scores work?'

results_from_pinecone = query_from_pinecone(query, top_k=5)

for result_from_pinecone in results_from_pinecone:
    print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")
    

1609fde30aed50a12485f7b137d2629a	0.84	Z-scores are an effective way to standardize  data
09f0a477c55e875f44a642d01a1ccf5c	0.84	We get this chart:
We can see that we have negativ
5458f5fa0f3cb04e25e79bf483f4bc31	0.84	Let's begin by learning a very  important value  i
268922260e7eabb5ed6170d07ec4585a	0.83	The preceding code gives us this graph:
Now, our d
68d05da324b87fb9c113dc598248dd4d	0.82	
Basic Statistics
This chapter will focus on the s


In [31]:
"""
This example computes the score between a query and all possible
sentences in a corpus using a Cross-Encoder for semantic textual similarity (STS).
It output then the most similar sentences for the given query.
"""
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np
from torch import nn

# Pre-trained cross encoder
# cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
# cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
cross_encoder = CrossEncoder('corrius/cross-encoder-mmarco-mMiniLMv2-L12-H384-v1')


/Users/jinkookchoi/dev/lecture/quick-start-guide-to-llms/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [32]:
def get_results_from_pinecone(query, top_k=3, re_rank=False, verbose=True):

    results_from_pinecone = query_from_pinecone(query, top_k=top_k)
    if not results_from_pinecone:
        return []

    if verbose:
        print("Query:", query)
    
    
    final_results = []

    if re_rank:
        if verbose:
            print('Document ID (Hash)\t\tRetrieval Score\tCE Score\tText')

        sentence_combinations = [[query, result_from_pinecone['metadata']['text']] for result_from_pinecone in results_from_pinecone]

        # Compute the similarity scores for these combinations
        similarity_scores = cross_encoder.predict(sentence_combinations, activation_fct=nn.Sigmoid())

        # Sort the scores in decreasing order
        sim_scores_argsort = reversed(np.argsort(similarity_scores))

        # Print the scores
        for idx in sim_scores_argsort:
            result_from_pinecone = results_from_pinecone[idx]
            final_results.append(result_from_pinecone)
            if verbose:
                print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{similarity_scores[idx]:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")
        return final_results

    if verbose:
        print('Document ID (Hash)\t\tRetrieval Score\tText')
    for result_from_pinecone in results_from_pinecone:
        final_results.append(result_from_pinecone)
        if verbose:
            print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")

    return final_results

In [33]:
final_results = get_results_from_pinecone(query, top_k=3, re_rank=True)

Query: How do z scores work?
Document ID (Hash)		Retrieval Score	CE Score	Text


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1609fde30aed50a12485f7b137d2629a	0.84	1.00	Z-scores are an effective way to standardize  data
5458f5fa0f3cb04e25e79bf483f4bc31	0.84	0.99	Let's begin by learning a very  important value  i
09f0a477c55e875f44a642d01a1ccf5c	0.84	0.14	We get this chart:
We can see that we have negativ


In [34]:
final_results = get_results_from_pinecone(query, top_k=10, re_rank=True)

Query: How do z scores work?
Document ID (Hash)		Retrieval Score	CE Score	Text


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1609fde30aed50a12485f7b137d2629a	0.84	1.00	Z-scores are an effective way to standardize  data
5458f5fa0f3cb04e25e79bf483f4bc31	0.84	0.99	Let's begin by learning a very  important value  i
68d05da324b87fb9c113dc598248dd4d	0.82	0.36	
Basic Statistics
This chapter will focus on the s
e1d5467e3658041aa16113edf0d3ebec	0.81	0.27	
This is no coincidence! When we standardize the d
09f0a477c55e875f44a642d01a1ccf5c	0.84	0.14	We get this chart:
We can see that we have negativ
66bbb15bf6a43d2b2077e7167face2bc	0.81	0.10	Summary
In this chapter, we covered much of the ba
268922260e7eabb5ed6170d07ec4585a	0.83	0.02	The preceding code gives us this graph:
Now, our d
75cf0bcd29070579356e3c50e9440d94	0.80	0.01	The following is the probability distribution of o
e9ec4bdefba2f80157ffce04812a1cf4	0.80	0.01	
Let's look at each of the elements in this formul
f90c995652f29ac9c2a4ca94f0e6809c	0.81	0.00	
We can think of this problem like as follows:
The


In [35]:
final_results[0]

{'id': '1609fde30aed50a12485f7b137d2629a',
 'metadata': {'date_uploaded': '2024-06-18T07:51:17.893437',
              'text': 'Z-scores are an effective way to standardize  data. '
                      'This means that we can put the entire set\n'
                      'on the same scale. For example, if we also measure each '
                      "person's general happiness scale\n"
                      '(which is between 0 and 1), we might have a dataset '
                      'similar to the following dataset:\n'
                      'friends = [109, 1017, 1127, 418, 625, 957, 89, 950, '
                      '946, 797, 981, 125,\n'
                      '455, 731, 1640, 485, 1309, 472, 1132, 1773, 906, 531, '
                      '742, 621]\n'
                      'happiness = [.8, .6, .3, .6, .6, .4, .8, .5, .4, .3, '
                      '.3, .6, .2, .8, 1, .6,\n'
                      '.2, .7, .5, .3, .1, 0, .3, 1]\n'
                      'import pandas as pd\n'
       

In [36]:
delete_texts_from_pinecone(pruned_documents)

{}

# BoolQ

In [1]:
from datasets import load_dataset
from evaluate import load

dataset = load_dataset("boolq")

/Users/jinkookchoi/dev/lecture/quick-start-guide-to-llms/.venv/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [5]:
len(dataset['train']), len(dataset['validation'])

(9427, 3270)

In [39]:
dataset['validation'][0]

{'question': 'does ethanol take more energy make that produces',
 'answer': False,
 'passage': "All biomass goes through at least some of these steps: it needs to be grown, collected, dried, fermented, distilled, and burned. All of these steps require resources and an infrastructure. The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance (or ``energy returned on energy invested''). Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in the US: one unit of fossil-fuel energy is required to create 1.3 energy units from the resulting ethanol. The energy balance for sugarcane ethanol produced in Brazil is more favorable, with one unit of fossil-fuel energy required to create 8 from the ethanol. Energy balance estimates are not easily produced, thus numerous such reports have been generated that are contradictory. For instance, a sep

In [40]:
for idx in tqdm(range(0, len(dataset['validation']), 256)):
    data_sample = dataset['validation'][idx:idx + 256]

    passages = data_sample['passage']

    upload_texts_to_pinecone(passages)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [01:16<00:00,  5.86s/it]


In [41]:
from random import sample

query = sample(dataset['validation']['question'], 1)[0]
print(query)
final_results = get_results_from_pinecone(query, top_k=3, re_rank=True)


do teams keep the fifa world cup trophy
Query: do teams keep the fifa world cup trophy
Document ID (Hash)		Retrieval Score	CE Score	Text


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

14b054eda54b77ea6eae9e9cce271e84	0.84	0.51	The World Cup is a gold trophy that is awarded to 
323ff6fa843c8f126857d4f1e5d3d5bf	0.83	0.43	The trophy has the engraving ``FIFA World Cup'' on
667210acd580564a6240d7ad5a20dd6a	0.83	0.42	The trophy has the engraving ``FIFA World Cup'' on


In [42]:
q_to_hash = {data['question']: my_hash(data['passage']) for data in dataset['validation']}

q_to_hash[query]

'667210acd580564a6240d7ad5a20dd6a'

In [53]:
# super_glue_metric = load('super_glue', 'boolq')  # just accuracy

# Let's test the performance re-ranking against 1000 of our validation datapoints
# Note we could not use Pinecone here to speed things up
#  but it's also a good time to test latency of the pipeline with Pinecone
val_sample = dataset['validation'][:1000]

In [55]:
logger.setLevel(logging.CRITICAL)

predictions = []

# Note we will keep top_k the same so latency from Pinecone is consistent
#  and the only major time difference will be in the re-ranking

for question in tqdm(val_sample['question']):
    # 쿼리 결과 해시 
    retrieved_hash = get_results_from_pinecone(question, top_k=1, re_rank=False, verbose=False)[0]['id']

    # 문제 해시 
    correct_hash = q_to_hash[question]

    # 같다면 True
    predictions.append(retrieved_hash == correct_hash)
    
accuracy = sum(predictions)/len(predictions)

print(f'Accuracy without re-ranking: {accuracy}')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:52<00:00,  2.12it/s]

Accuracy without re-ranking: 0.831


In [99]:
logger.setLevel(logging.CRITICAL)

predictions = []

# Note we will keep top_k the same so latency from Pinecone is consistent
#  and the only major time difference will be in the re-ranking

for question in tqdm(val_sample['question']):
    # re_rank = True 설정
    # 설정 여부에 따라 소요 시간 차이가 날 듯
    retrieved_hash = get_results_from_pinecone(question, top_k=3, re_rank=True, verbose=False)[0]['id']
    correct_hash = q_to_hash[question]
    predictions.append(retrieved_hash == correct_hash)
    
accuracy = sum(predictions)/len(predictions)

print(f'Accuracy with re-ranking: {accuracy}')

100%|██████████████████████████████████████████████████████████████████████| 3270/3270 [27:20<00:00,  1.99it/s]

Accuracy with re-ranking: 0.8373088685015291


In [100]:
# Note the time differences between with and without re-ranking


In [56]:
def eval_ranking(query, cross_encoder, top_k=3):
    results_from_pinecone = query_from_pinecone(query, top_k=top_k)
    sentence_combinations = [[query, result_from_pinecone['metadata']['text']] for result_from_pinecone in results_from_pinecone]
    similarity_scores = cross_encoder.predict(sentence_combinations)
    sim_scores_argsort = list(reversed(np.argsort(similarity_scores)))
    re_ranked_final_result = results_from_pinecone[sim_scores_argsort[0]]
    return results_from_pinecone[0]['id'], re_ranked_final_result['id']


In [57]:
# Trying another pre-trained cross encoder
# sentence-transformers/multi-qa-mpnet-base-cos-v1
newer_cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')


/Users/jinkookchoi/dev/lecture/quick-start-guide-to-llms/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [58]:
i = 0
print_every = 50
predictions = []
for question in tqdm(val_sample['question']):
    retrieved_hash, reranked_hash = eval_ranking(question, newer_cross_encoder, top_k=3)
    correct_hash = q_to_hash[question]
    predictions.append((retrieved_hash == correct_hash, reranked_hash == correct_hash))
    i += 1
    if i % print_every == 0:
        print(f'Step {i}')
        raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
        reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

        print(f'Accuracy without re-ranking: {raw_accuracy}')
        print(f'Accuracy with re-ranking: {reranked_accuracy}')


  5%|████████████▋                                                                                                                                                                                                                                                | 50/1000 [00:36<11:45,  1.35it/s]

Step 50
Accuracy without re-ranking: 0.88
Accuracy with re-ranking: 0.84


 10%|█████████████████████████▏                                                                                                                                                                                                                                  | 100/1000 [01:13<11:15,  1.33it/s]

Step 100
Accuracy without re-ranking: 0.85
Accuracy with re-ranking: 0.85


 15%|█████████████████████████████████████▊                                                                                                                                                                                                                      | 150/1000 [01:53<11:59,  1.18it/s]

...

Step 400
Accuracy without re-ranking: 0.865
Accuracy with re-ranking: 0.8475


 45%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                          | 450/1000 [05:40<06:05,  1.50it/s]

Step 450
Accuracy without re-ranking: 0.86
Accuracy with re-ranking: 0.8422222222222222


 49%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                               | 494/1000 [06:09<06:18,  1.34it/s]


KeyboardInterrupt: 

In [ ]:
raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

print(f'Using cross-encoder: {newer_cross_encoder.config._name_or_path}')
print(f'Accuracy without re-ranking: {raw_accuracy}')
print(f'Accuracy with re-ranking: {reranked_accuracy}')


# Fine-tuning re-ranker

In [59]:
# https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/ms_marco/train_cross-encoder_scratch.py

In [60]:
dataset['train'][0]

{'question': 'do iran and afghanistan speak the same language',
 'answer': True,
 'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.'}

In [61]:
dataset['train'][1]

{'question': 'do good samaritan laws protect those who help at an accident',
 'answer': True,
 'passage': "Good Samaritan laws offer legal protection to people who give reasonable assistance to those who are, or who they believe to be, injured, ill, in peril, or otherwise incapacitated. The protection is intended to reduce bystanders' hesitation to assist, for fear of being sued or prosecuted for unintentional injury or wrongful death. An example of such a law in common-law areas of Canada: a good Samaritan doctrine is a legal principle that prevents a rescuer who has voluntarily helped a victim in distress from being successfully sued for wrongdoing. Its purpose is to keep people from being reluctant to help a stranger in need for fear of legal repercussions should they make some mistake in treatment. By contrast, a duty to rescue law requires people to offer assistance and holds those who fail to do so liable."}

In [62]:
from sentence_transformers import InputExample, losses, evaluation
from torch.utils.data import DataLoader
from random import shuffle

shuffled_training_passages = dataset['train']['passage'].copy()
shuffle(shuffled_training_passages)


train_samples = [
  InputExample(texts=[d['question'], d['passage']], label=1) for d in dataset['train']
]

# add some negative examples
train_samples += [
  InputExample(texts=[d['question'], shuffled_training_passages[i]], label=0) for i, d in enumerate(dataset['train'])
]

shuffle(train_samples)

# running the risk of overfitting on my data but maybe I want that. 
#  Combined with sufficient input and output validation, we can make a viable product with a model overfit to my data


In [63]:
len(train_samples)

18854

In [64]:
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', num_labels=1)

In [65]:
train_samples[0].__dict__

{'guid': '',
 'texts': ['was the laramide orogeny related to subduction of an oceanic plateau',
  'The calendar is a lunar calendar, and months begin when the first crescent of a new moon is sighted. Since the Islamic lunar calendar year is 10 to 11 days shorter than the solar year and contains no intercalation, Ramadan migrates throughout the seasons. The Islamic day starts after sunset. The estimated start and end dates for Ramadan, based on the Umm al-Qura calendar of Saudi Arabia, are:'],
 'label': 0}

In [66]:
model.predict(train_samples[0].texts, activation_fct=nn.Sigmoid())

1.288899e-05

In [67]:
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator, CEBinaryClassificationEvaluator
import math
import torch
from random import sample

logger.setLevel(logging.DEBUG)  # just to get some logs

num_epochs = 2

model_save_path = './fine_tuned_ir_cross_encoder'

# train_samples = sample(train_samples, 1000)

# int(len(train_samples)*.8)
train_dataloader = DataLoader(train_samples[:int(len(train_samples)*.8)], shuffle=True, batch_size=32)

# An evaluator for training performance
evaluator = CEBinaryClassificationEvaluator.from_input_examples(train_samples[-int(len(train_samples)*.8):], name='test')

# Rule of thumb for warmup steps
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
print(f"Warmup-steps: {warmup_steps}")

Warmup-steps: 95


In [ ]:
# # ##### Load model and eval on test set
# print(evaluator(model))

# Train the model
model.fit(
    train_dataloader=train_dataloader,
    loss_fct=losses.nn.CrossEntropyLoss(),
    activation_fct=nn.Sigmoid(),
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    use_amp=True
)

# ##### Load model and eval on test set
# print(evaluator(model))


In [ ]:
# run the more fine tuned version on open source as well to match??
# depends if it does better here

In [359]:
finetuned = CrossEncoder(model_save_path)

print(finetuned.predict(['hello', 'hi'], activation_fct=nn.Sigmoid()))
print(finetuned.predict(['hello', 'hi'], activation_fct=nn.Identity()))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.9999926


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

11.819853


In [363]:
# Trying our fine-tuned cross encoder
logger.setLevel(logging.CRITICAL)  # just to suppress some logs
from tqdm import tqdm

i = 0
print_every = 50
predictions = []
for question in tqdm(val_sample['question']):
    retrieved_hash, reranked_hash = eval_ranking(question, finetuned, top_k=3)
    correct_hash = q_to_hash[question]
    predictions.append((retrieved_hash == correct_hash, reranked_hash == correct_hash))
    i += 1
    if i % print_every == 0:
        print(f'Step {i}')
        raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
        reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

        print(f'Accuracy without re-ranking: {raw_accuracy}')
        print(f'Accuracy with re-ranking: {reranked_accuracy}')


  2%|█                                                                       | 51/3270 [00:44<19:37,  2.73it/s]

Step 50
Accuracy without re-ranking: 0.88
Accuracy with re-ranking: 0.82


  3%|██▏                                                                    | 100/3270 [01:07<23:44,  2.23it/s]

Step 100
Accuracy without re-ranking: 0.85
Accuracy with re-ranking: 0.84


  5%|███▎                                                                   | 150/3270 [01:32<24:54,  2.09it/s]

...

 98%|████████████████████████████████████████████████████████████████████▌ | 3201/3270 [27:31<00:20,  3.36it/s]

Step 3200
Accuracy without re-ranking: 0.8525
Accuracy with re-ranking: 0.83875


 99%|█████████████████████████████████████████████████████████████████████▌| 3250/3270 [27:50<00:05,  3.50it/s]

Step 3250
Accuracy without re-ranking: 0.8526153846153847
Accuracy with re-ranking: 0.8384615384615385


100%|██████████████████████████████████████████████████████████████████████| 3270/3270 [27:57<00:00,  1.95it/s]


In [364]:
# Re-ranking got slightly better after 2 epochs.

In [120]:
raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

print(f'Using cross-encoder: {finetuned.config._name_or_path}')
print(f'Accuracy without re-ranking: {raw_accuracy}')
print(f'Accuracy with re-ranking: {reranked_accuracy}')


Using cross-encoder: <sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder object at 0x158c9cc70>
Accuracy without re-ranking: 0.8522935779816514
Accuracy with re-ranking: 0.8495412844036697


In [122]:
# pinecone.delete_index(INDEX_NAME)  # delete the index

# OPEN SOURCE ALTERNATIVE TO EMBEDDING

In [70]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

docs = ["Around 9 Million people live in London", "London is known for its financial district"]

doc_emb = model.encode(docs, batch_size=32, show_progress_bar=True)

doc_emb.shape#  == ('2, 768')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(2, 768)

In [379]:
#Encode query and documents
docs = dataset['validation']['passage']
doc_emb = model.encode(docs, batch_size=32, show_progress_bar=True)

100%|██████████████████████████████████████████████████████████████████████████| 33/33 [00:29<00:00,  1.11it/s]


In [380]:
from random import sample

query = sample(dataset['validation']['question'], 1)[0]
print(query)
final_results = get_results_from_pinecone(query, top_k=3, re_rank=True)


did greg oden win a championship with miami
Query: did greg oden win a championship with miami
Document ID (Hash)		Retrieval Score	CE Score	Text
dc737eb4be84a308b945fb697a67f922	0.82	0.35	On January 15, 2014, Oden made his long-awaited re
275270d2171ffe4d024015ab4ea3343b	0.78	0.05	Selected 5th overall in the 2003 NBA draft by the 
ed060ec9bb1129f14f06186a2c5d29b4	0.78	0.02	The 2012 NBA Finals was the championship series of


In [381]:
query_emb = model.encode(query)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs[:3]:
    print(score, doc)


0.8174281120809472 On January 15, 2014, Oden made his long-awaited return to the court. In his first regular season game since December 2009, he recorded 6 points and 2 rebounds in 8 minutes of game time in a 114--97 loss to the Washington Wizards. On February 23, 2014, Oden made his first start since December 2009 in the Heat's 93--79 win over the Chicago Bulls. The Heat made the 2014 NBA Finals where they faced the San Antonio Spurs. They went on to lose the series in five games.
0.7841596747078865 Selected 5th overall in the 2003 NBA draft by the Miami Heat, Wade quickly emerged as a productive player on a youthful Miami Heat team and averaged 16.2 points on 46.5% shooting with averages of 4.0 rebounds and 4.5 assists per game. Wade is one of only four Marquette University players to be drafted in the first round; his is the highest draft selection in school history. After a 5--15 start, the Heat would gradually improve and finish 42--40 to qualify for the NBA playoffs. He further d

In [382]:
logger.setLevel(logging.CRITICAL)  # just to suppress some logs


def eval_ranking_open_source(query, cross_encoder, top_k=3):
    query_emb = np.array(get_embedding(query, engine=ENGINE))

    #Compute dot score between query and all document embeddings
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

    #Combine docs & scores
    doc_score_pairs = list(zip(docs, scores))

    #Sort by decreasing score
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)[:top_k]

    retrieved_hash = my_hash(doc_score_pairs[0][0])
    if cross_encoder:
        sentence_combinations = [[query, doc_score_pair[0]] for doc_score_pair in doc_score_pairs]
        similarity_scores = cross_encoder.predict(sentence_combinations)
        sim_scores_argsort = list(reversed(np.argsort(similarity_scores)))
        reranked_hash = my_hash(doc_score_pairs[sim_scores_argsort[0]][0])
    else:
        reranked_hash = None
    return retrieved_hash, reranked_hash


In [383]:
eval_ranking_open_source(query, finetuned)

('dc737eb4be84a308b945fb697a67f922', 'dc737eb4be84a308b945fb697a67f922')

In [384]:
logger.setLevel(logging.CRITICAL)

i = 0
print_every = 50
predictions = []
for question in tqdm(val_sample['question']):
    retrieved_hash, reranked_hash = eval_ranking_open_source(question, finetuned, top_k=3)
    correct_hash = q_to_hash[question]
    predictions.append((retrieved_hash == correct_hash, reranked_hash == correct_hash))
    i += 1
    if i % print_every == 0:
        print(f'Step {i}')
        raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
        reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

        print(f'Accuracy without re-ranking: {raw_accuracy}')
        print(f'Accuracy with re-ranking: {reranked_accuracy}')


  2%|█                                                                       | 51/3270 [00:16<11:01,  4.86it/s]

Step 50
Accuracy without re-ranking: 0.88
Accuracy with re-ranking: 0.84


  3%|██▏                                                                    | 100/3270 [00:31<14:50,  3.56it/s]

Step 100
Accuracy without re-ranking: 0.85
Accuracy with re-ranking: 0.85


  5%|███▎                                                                   | 151/3270 [00:49<10:45,  4.83it/s]

...

 98%|████████████████████████████████████████████████████████████████████▌ | 3201/3270 [15:21<00:17,  4.06it/s]

Step 3200
Accuracy without re-ranking: 0.8525
Accuracy with re-ranking: 0.8425


 99%|█████████████████████████████████████████████████████████████████████▌| 3251/3270 [15:31<00:03,  5.11it/s]

Step 3250
Accuracy without re-ranking: 0.8529230769230769
Accuracy with re-ranking: 0.8427692307692307


100%|██████████████████████████████████████████████████████████████████████| 3270/3270 [15:35<00:00,  3.49it/s]


In [385]:
raw_accuracy = sum([p[0] for p in predictions])/len(predictions)
reranked_accuracy = sum([p[1] for p in predictions])/len(predictions)

print(f'Using cross-encoder: {finetuned.config._name_or_path}')
print(f'Accuracy without re-ranking: {raw_accuracy}')
print(f'Accuracy with re-ranking: {reranked_accuracy}')


Using cross-encoder: cross-encoder/mmarco-mMiniLMv2-L12-H384-v1
Accuracy without re-ranking: 0.8525993883792049
Accuracy with re-ranking: 0.8434250764525993
